In [1]:
from dask.distributed import Client
import dask.dataframe as dd
import pandas as pd
import os
import shutil
import sys
import logging
import glob
import numpy as np
from itertools import product
import gc

#import pyarrow

In [2]:
client = Client("tcp://127.0.0.1:9100")

In [3]:
client

<Client: 'tcp://127.0.0.1:9100' processes=4 threads=4, memory=18.63 GiB>

In [4]:
os.getcwd()

'/mnt/labshares/schumm-lab/bryanb/teststate/scripts'

In [5]:
# NOTE: July 2022: not all states have rx files, so I've selected a state that does
# NOTE: rx files do not have the "XXoccr" file that other sections do. Just a line and header.
state_dir="../../data2/medicaid/2016/CT/taf"

In [6]:
# name of fake state (set in create_state.sh)
new_state_name="TS2"

# section files (CHANGE THESE TO GENERATE DIFFERENT SECTIONS)
section='rx'
filenames=['rxh', 'rxl']

section_dir=f'{state_dir}/{section}'

# Create empty dataframes with file names
file_vars=[]
for file in filenames:
    vars()[file] = pd.DataFrame()
    file_vars.append(vars()[file])
    
# Add medicaid data to the dataframes
for i in range(len(file_vars)):
    file_vars[i]=pd.read_parquet(f'{section_dir}/{filenames[i]}/parquet/part.0.parquet', engine = 'pyarrow')

In [7]:
# Drop data and create empty dataframes containing just the columns
for i in range(len(file_vars)):
    vars()[filenames[i] + '_empty']=file_vars[i].drop(file_vars[i][file_vars[i].index >= '0'].index)
    print(f'Testing {filenames[i] + "_empty"} is empty:')
    assert(vars()[filenames[i] + '_empty'].empty)
    
# save files as empty fake state data
for i in range(len(file_vars)):
    print(f'Saving {filenames[i] + "_empty"} as {filenames[i] + ".csv"}')
    vars()[filenames[i] + "_empty"].to_csv(f'{filenames[i]}.csv')


Testing rxh_empty is empty:
Testing rxl_empty is empty:
Saving rxh_empty as rxh.csv
Saving rxl_empty as rxl.csv


In [8]:
# move files to their appropriate location
for filename in filenames:
    shutil.move(f'{filename}.csv', f'../{new_state_name}/taf/{section}/{filename}/csv/{filename}.csv')
    print(f'moving {filename} to ../{new_state_name}/taf/{section}/{filename}/csv/{filename}.csv')

moving rxh to ../TS2/taf/rx/rxh/csv/rxh.csv
moving rxl to ../TS2/taf/rx/rxl/csv/rxl.csv
